### Project 1

We need to design an dimplement a class that will be used to represent bank accounts.

We want the following functionality and characteristics:
- accounts are uniquely identified by an **account number** (assume it will just be passed in the initializer)
- account holders have a **first** and **last** name
- accounts have an associated **preferred time zone offset** (e.g. -7 for MST)
- **balances** need to be zero or higher, and should not be directly settable.
- but, **deposits** and **withdrawals** can be made (given sufficient funds)
    - if a withdrawal is attempted that would result in nagative funds, the transaction should be declined.
- a **monthly interest rate** exists and is applicable to all accounts **uniformly**. There should be a method that can be called to calculate the interest on the current balance using the current interest rate, and **add it** to the balance.
- each deposit and withdrawal must generate a **confirmation number** composed of:
    - the transaction type: `D` for deposit, and `W` for withdrawal, `I` for interest deposit, and `X` for declined (in which case the balance remains unaffected)
    - the account number
    - the time the transaction was made, using UTC
    - an incrementing number (that increments across all accounts and transactions)
    - for (extreme!) simplicity assume that the transaction id starts at zero (or whatever number you choose) whenever the program starts
    - the confirmation number should be returned from any of the transaction methods (deposit, withdraw, etc)
- create a **method** that, given a confirmation number, returns:
    - the account number, transaction code (D, W, etc), datetime (UTC format), date time (in whatever timezone is specified in te argument, but more human readable), the transaction ID
    - make it so it is a nicely structured object (so can use dotted notation to access these three attributes)
    - I purposefully made it so the desired timezone is passed as an argument. Can you figure out why? (hint: does this method require any information from any instance?)

For example, we may have an account with:
- account number `140568` 
- preferred time zone offset of -7 (MST) 
- an existing balance of `100.00`

Suppose the last transaction ID in the system was `123`, and a deposit is made for `50.00` on `2019-03-15T14:59:00` (UTC) on that account (or `2019-03-15T07:59:00` in account's preferred time zone offset)

The new balance should reflect `150.00` and the confirmation number returned should look something like this:

```D-140568-20190315145900-124```

We also want a method that given the confirmation number returns an object with attributes:
- `result.account_number` --> `140568`
- `result.transaction_code` --> `D`
- `result.transaction_id` --> `124`
- `result.time` --> `2019-03-15 07:59:00 (MST)`
- `result.time_utc` --> `2019-03-15T14:59:00`

Furthermore, if current interest rate is `0.5%`, and the account's balance is `1000.00`, then the result of calling the `deposit_interest` (or whatever name you choose) method, should result in a new transaction and a new balance of `1050.00`. Calling this method should also return a confirmation number.

For simplicty, just use floats, but be aware that for these types of situations you'll probably want to use `Decimal` objects instead of floats.

There are going to be many ways to design something like this, especially since I have not nailed down all the specific requirements, so you'll have to fill the gaps yourself and decide what other things you may want to implement (like is the account number going to be a mutable property, or "read-only" and so on).

See how many different ideas you can use from what we covered in the last section. 

My approach will end up creating two classes: a `TimeZone` class used to store the time zone name and offset definition (in hours and minutes), and a main class called `Account` that will have the following "public" interface:
- initializer with account number, first name, last name, optional preferred time zone, starting balance (defaults to 0)
- a first name property (read/write)
- a last name property (read/write)
- a full name property (computed, read-only)
- a balance property (read-only)
- an interest rate property (class level property)
- deposit, withdraw, pay_interest methods
- parse confirmation code

Class will have additional state and methods, but those will be used for implementation.

You should also remember to test your code! In the solutions I will introduce you to Python's `unittest` package. Even if you skip this project, at least review that video and/or notebook if you are unfamiliar with `unittest`.

In [30]:
import numbers
from datetime import datetime, timezone, timedelta

class CustTimeZone:

    def __init__(self,name,offset_hours,offset_minutes):
        if name is None or len(str(name).strip())==0:
            raise ValueError('Timezone name cannot be empty')
        
        self._name = name.strip()
        if not isinstance(offset_hours, numbers.Integral):
            raise ValueError('Timezone offset must be an integer')
        
        if not isinstance(offset_minutes, numbers.Integral):
            raise ValueError('Timezone offset must be an integer')
        
        if offset_minutes>59 or offset_minutes <-59:
            raise ValueError('Minutes offset must be between -59 and 59 inclusive')

        offset = timedelta(hours=offset_hours, minutes=offset_minutes)
        if offset < timedelta(hours=-12, minutes=0) or offset > timedelta(hours=14, minutes=0):
            raise ValueError('Offset must be between -12:00 and +14:00')
        
        self._offset_hours = offset_hours
        self.offset_minutes = offset_minutes
        self._offset = offset

    @property
    def offset(self):
        return self._offset
    
    @property
    def name(self):
        return self._name
    
    def __eq__(self,other):
        return (isinstance(other, CustTimeZone) and 
        self.name == other.name and 
        self._offset_hours == other._offset_hours and
        self._offset_minutes == other._offset_minutes)
    
    def __repr__(self):
        return (f"TimeZone(name='{self.name})',"
                f"offset_hours ={self.offset_hours}',"
                f"offset_minutes = '{self.offset_minutes}'")

In [34]:
cust_time_zone = CustTimeZone('ABC',-2,-15)
cust_time_zone.name
dt = datetime.utcnow()
print(dt)

print(dt+cust_time_zone.offset)



2024-07-13 19:52:04.901334
2024-07-13 17:37:04.901334


In [36]:
class TransactionID:
    def __init__(self,start_id):
        self._start_id = start_id
    def next(self):
        self._start_id += 1
        return self._start_id

In [40]:
def transaction_id(start_id):
    start_id+=1
    yield start_id

In [51]:
from collections import namedtuple

Confirmation = namedtuple('Confirmation','account_number transaction_code transaction_id time_utc time')

class Account:
    _interest_rate = 0.05
    transaction_manager = transaction_id(1)

    _transaction_codes = {
        'deposit':'D',
        'withdraw': 'W',
        'interest': 'I',
        'rejected': 'X'
    }

    def __init__(self,account_number,first_name,last_name,
                 timezone=None,initial_balance=0):
        self._account_number = account_number
        self.first_name = first_name
        self.last_name = last_name
        self._balance = float(initial_balance)

        if timezone is None:
            timezone = CustTimeZone('UTC',0,0)
        self.timezone = timezone

        if initial_balance<0:
            raise ValueError("inital balance cant be negative")


    def make_transaction(self):
        return Account.transaction_manager.next()
    
    @staticmethod
    def validate_name(value, field_title):
        if value is None or len(value.strip())==0:
            raise ValueError(f"{field_title} name cant be empty")
        return str(value).strip()

    @property
    def first_name(self):
        return self._first_name
    
    @first_name.setter
    def first_name(self,value):
        self._first_name = Account.validate_name(value,'first_name')
    
    @property
    def last_name(self):
        return self._last_name
    
    @last_name.setter
    def last_name(self,value):
        self._last_name = Account.validate_name(value,'last_name')
    
    @property
    def full_name(self):
        return " ".join([self._first_name,self._last_name])
    
    @property
    def balance(self):
        return self._balance
    
    @property
    def timezone(self):
        return self._timezone
    
    @timezone.setter
    def timezone(self,value):
        if not isinstance(value,CustTimeZone):
            raise ValueError('Time Zone must be a valid TimeZone object')
        self._timezone = value

    def get_interest_rate(self):
        return Account.interest_rate
    
    def set_interest_rate(self,value):
        if not isinstance(value, numbers.Real):
            raise ValueError('interest rate must be a real number.')
        
        if value<0:
            raise ValueError('interest rate cant be a negative value.')
        
        Account._interest_rate = value

    def generate_confirmation_code(self,transaction_code):
        transaction_dt = datetime.utcnow().strftime('%Y%m%d%H%M%S')
        transaction_id = Account.transaction_manager.next()
        confirmation_code = "-".join([transaction_code,self.account_number,transaction_dt,transaction_id])
        return confirmation_code

    def deposit(self,deposit_value):
        transaction_code= Account._transaction_codes.get('deposit')
        if deposit_value <0:
            raise ValueError('Deposit value cant be a negative value')
        #incremementar balance
        self._balance += deposit_value
        # confirmation_code
        self.generate_confirmation_code(self,transaction_code)
  

    def withdraw(self,withdraw_value):
        transaction_code= Account._transaction_codes.get('withdraw')
        
        if withdraw_value > self._balance:
            raise ValueError('Withraw value cant be a greater than balance')
        #incremementar balance
        self._balance -= withdraw_value
        # confirmation_code
        self.generate_confirmation_code(self,transaction_code)

    def pay_interest(self):
        transaction_code= Account._transaction_codes.get('interest')
        #incremementar balance
        self._balance += self._balance * Account.INTEREST_RATE
        # confirmation_code
        self.generate_confirmation_code(self,transaction_code)
    
    @staticmethod
    def get_information(confirmation_code, preferred_time_zone=None):

        attributes = list(confirmation_code.split("-"))

        if len(attributes)!=4:
            raise ValueError('Invalid confirmation code')
        
        transaction_code, account_number, raw_dt_utc,transaction_id = attributes
        
        try:
            dt_utc = datetime.strptime(raw_dt_utc,'%Y%m%d%H%M%S')
        except ValueError as ex:
            raise ValueError('Invalid transaction datetime') from ex
        
        if preferred_time_zone is None:
            preferred_time_zone = CustTimeZone('UTC',0,0)
        if not isinstance(preferred_time_zone,CustTimeZone):
            raise ValueError('Invalid TimeZone specified.')
        
        dt_preferred_time_zone = dt_utc+ preferred_time_zone.offset
        
        Confirmation(transaction_code, account_number, raw_dt_utc,transaction_id )

In [47]:
cust_account = Account('1234','lina','tobon','-7:00',balance=0)

ValueError: Time Zone must be a valid TimeZone object

In [52]:
cust_account = Account('1234','lina','tobon',balance=0)